In [196]:
#East Side vs West Side Restaurants
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from sqlalchemy import create_engine
import os
import csv
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import numpy as np
import requests
import time
import csv
import time
from datetime import datetime
import seaborn
from scipy import stats
from flask import json

In [197]:
#*************************************-----------WEST SIDE DATA------------------*****************************************

In [198]:
#verify data is pulled into jupyter
#LA data is west coast
westside_file = "../Resources/west_side.csv"
westside_data_df = pd.read_csv(westside_file)
#show data
westside_data_df.head(5)

,activity_date,employee_id,facility_address,facility_city,facility_id,facility_name,facility_state,facility_zip,grade,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id,score,serial_number,service_code,service_description
0,5/9/2017,EE0000593,17660 CHATSWORTH ST,GRANADA HILLS,FA0175397,HOVIK'S FAMOUS MEAT & DELI,CA,91344,A,OW0181955,JOHN'S FAMOUS MEAT & DELI INC.,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,HOVIK'S FAMOUS MEAT & DELI,ACTIVE,PR0168541,98,DAHDRUQZO,1,ROUTINE INSPECTION
1,4/10/2017,EE0000126,3615 PACIFIC COAST HWY,TORRANCE,FA0242138,SHAKEY'S PIZZA,CA,90505,A,OW0237843,"SCO, LLC",RESTAURANT (61-150) SEATS HIGH RISK,1638,SHAKEY'S PIZZA,ACTIVE,PR0190290,94,DAL3SBUE0,1,ROUTINE INSPECTION
2,4/4/2017,EE0000593,17515 CHATSWORTH ST,GRANADA HILLS,FA0007801,BAITH AL HALAL,CA,91344,A,OW0031150,SABIR MOHAMMAD SHAHID,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,BAITH AL HALAL,INACTIVE,PR0036723,95,DAL2PIKJU,1,ROUTINE INSPECTION
3,8/15/2017,EE0000971,44455 VALLEY CENTRAL WAY,LANCASTER,FA0013858,FOOD 4 LESS #306,CA,93536,A,OW0012108,"FOOD 4 LESS, INC.",RESTAURANT (0-30) SEATS HIGH RISK,1632,FOOD 4 LESS DELI/BAKERY#306,ACTIVE,PR0039905,98,DA0ZMAJXZ,1,ROUTINE INSPECTION
4,9/26/2016,EE0000145,11700 SOUTH ST,ARTESIA,FA0179671,PHO LITTLE SAIGON,CA,90701,A,OW0185167,PHO SOUTH ST INC,RESTAURANT (61-150) SEATS HIGH RISK,1638,PHO LITTLE SAIGON,ACTIVE,PR0173311,96,DA41DBXA2,1,ROUTINE INSPECTION


In [199]:
#get year from date
westside_data_df["Year"]=pd.DatetimeIndex(westside_data_df["activity_date"]).year
westside_data_df.head(5)

,activity_date,employee_id,facility_address,facility_city,facility_id,facility_name,facility_state,facility_zip,grade,owner_id,...,pe_description,program_element_pe,program_name,program_status,record_id,score,serial_number,service_code,service_description,Year
0,5/9/2017,EE0000593,17660 CHATSWORTH ST,GRANADA HILLS,FA0175397,HOVIK'S FAMOUS MEAT & DELI,CA,91344,A,OW0181955,...,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,HOVIK'S FAMOUS MEAT & DELI,ACTIVE,PR0168541,98,DAHDRUQZO,1,ROUTINE INSPECTION,2017
1,4/10/2017,EE0000126,3615 PACIFIC COAST HWY,TORRANCE,FA0242138,SHAKEY'S PIZZA,CA,90505,A,OW0237843,...,RESTAURANT (61-150) SEATS HIGH RISK,1638,SHAKEY'S PIZZA,ACTIVE,PR0190290,94,DAL3SBUE0,1,ROUTINE INSPECTION,2017
2,4/4/2017,EE0000593,17515 CHATSWORTH ST,GRANADA HILLS,FA0007801,BAITH AL HALAL,CA,91344,A,OW0031150,...,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,BAITH AL HALAL,INACTIVE,PR0036723,95,DAL2PIKJU,1,ROUTINE INSPECTION,2017
3,8/15/2017,EE0000971,44455 VALLEY CENTRAL WAY,LANCASTER,FA0013858,FOOD 4 LESS #306,CA,93536,A,OW0012108,...,RESTAURANT (0-30) SEATS HIGH RISK,1632,FOOD 4 LESS DELI/BAKERY#306,ACTIVE,PR0039905,98,DA0ZMAJXZ,1,ROUTINE INSPECTION,2017
4,9/26/2016,EE0000145,11700 SOUTH ST,ARTESIA,FA0179671,PHO LITTLE SAIGON,CA,90701,A,OW0185167,...,RESTAURANT (61-150) SEATS HIGH RISK,1638,PHO LITTLE SAIGON,ACTIVE,PR0173311,96,DA41DBXA2,1,ROUTINE INSPECTION,2016


In [200]:
#add column and add location
westside_datalocation_df=westside_data_df.assign(Location="Westside")
westside_datalocation_df.head(5)

,activity_date,employee_id,facility_address,facility_city,facility_id,facility_name,facility_state,facility_zip,grade,owner_id,...,program_element_pe,program_name,program_status,record_id,score,serial_number,service_code,service_description,Year,Location
0,5/9/2017,EE0000593,17660 CHATSWORTH ST,GRANADA HILLS,FA0175397,HOVIK'S FAMOUS MEAT & DELI,CA,91344,A,OW0181955,...,1612,HOVIK'S FAMOUS MEAT & DELI,ACTIVE,PR0168541,98,DAHDRUQZO,1,ROUTINE INSPECTION,2017,Westside
1,4/10/2017,EE0000126,3615 PACIFIC COAST HWY,TORRANCE,FA0242138,SHAKEY'S PIZZA,CA,90505,A,OW0237843,...,1638,SHAKEY'S PIZZA,ACTIVE,PR0190290,94,DAL3SBUE0,1,ROUTINE INSPECTION,2017,Westside
2,4/4/2017,EE0000593,17515 CHATSWORTH ST,GRANADA HILLS,FA0007801,BAITH AL HALAL,CA,91344,A,OW0031150,...,1612,BAITH AL HALAL,INACTIVE,PR0036723,95,DAL2PIKJU,1,ROUTINE INSPECTION,2017,Westside
3,8/15/2017,EE0000971,44455 VALLEY CENTRAL WAY,LANCASTER,FA0013858,FOOD 4 LESS #306,CA,93536,A,OW0012108,...,1632,FOOD 4 LESS DELI/BAKERY#306,ACTIVE,PR0039905,98,DA0ZMAJXZ,1,ROUTINE INSPECTION,2017,Westside
4,9/26/2016,EE0000145,11700 SOUTH ST,ARTESIA,FA0179671,PHO LITTLE SAIGON,CA,90701,A,OW0185167,...,1638,PHO LITTLE SAIGON,ACTIVE,PR0173311,96,DA41DBXA2,1,ROUTINE INSPECTION,2016,Westside


In [201]:
#need only date, business name and inspection grade
westside_new_df= westside_datalocation_df[["Year", "facility_name", "grade", "Location"]]
westside_new_df.head()

##chicago_crime = chicago_crime_df[["Case Number", "Date", "Description"]]
##chicago_crime.head(5)

,Year,facility_name,grade,Location
0,2017,HOVIK'S FAMOUS MEAT & DELI,A,Westside
1,2017,SHAKEY'S PIZZA,A,Westside
2,2017,BAITH AL HALAL,A,Westside
3,2017,FOOD 4 LESS #306,A,Westside
4,2016,PHO LITTLE SAIGON,A,Westside


In [202]:
#drop blank data
westside_drop_df=westside_new_df.dropna(how='any', axis=0)
westside_drop_df.head(5)

,Year,facility_name,grade,Location
0,2017,HOVIK'S FAMOUS MEAT & DELI,A,Westside
1,2017,SHAKEY'S PIZZA,A,Westside
2,2017,BAITH AL HALAL,A,Westside
3,2017,FOOD 4 LESS #306,A,Westside
4,2016,PHO LITTLE SAIGON,A,Westside


In [203]:
#selecting dates 2015,2016,2017
date_range=(westside_drop_df['Year']>2014) & (westside_drop_df['Year']<2018)
westside_final_df=westside_drop_df[date_range]
westside_final_df.tail(10)

,Year,facility_name,grade,Location
191361,2017,WHOLE FOODS MARKET #20,A,Westside
191362,2016,LOUISIANA FRIED CHICKEN #380,A,Westside
191363,2016,JACK IN THE BOX #354,A,Westside
191364,2016,POQUITO MAS,A,Westside
191365,2017,STATER BROS #137,A,Westside
191366,2016,VONS MARKET,A,Westside
191367,2016,MOANA GRILL,A,Westside
191368,2016,QUEST CAFE,A,Westside
191369,2016,PINA'S MEXICAN RESTAURANT,A,Westside
191370,2017,RINCON HODURENO,A,Westside


In [204]:
#data statistics--ALL
west_final_stats_df=westside_final_df.describe(include='all')
west_final_stats_df


,Year,facility_name,grade,Location
count,191371.000000,191371,191371,191371
unique,NaN,36627,4,1
top,NaN,SUBWAY,A,Westside
freq,NaN,576,181278,191371
mean,2016.212341,NaN,NaN,NaN
std,0.732752,NaN,NaN,NaN
min,2015.000000,NaN,NaN,NaN
25%,2016.000000,NaN,NaN,NaN
50%,2016.000000,NaN,NaN,NaN
75%,2017.000000,NaN,NaN,NaN


In [205]:
#counts by grade
#west_final_group_df.rename(index=str, columns={"facility_name":"Business", "grade":"GRADE"}, inplace=True)
west_final_group_df=westside_final_df.groupby('grade').count()
west_final_group_df

,Year,facility_name,Location
grade,,,
,32,32,32
A,181278,181278,181278
B,9379,9379,9379
C,682,682,682


In [206]:
#Rename columns
westside_final_df.rename(index=str, columns={"facility_name":"Business", "grade":"GRADE"}, inplace=True)
westside_final_df.head(5)

,Year,Business,GRADE,Location
0,2017,HOVIK'S FAMOUS MEAT & DELI,A,Westside
1,2017,SHAKEY'S PIZZA,A,Westside
2,2017,BAITH AL HALAL,A,Westside
3,2017,FOOD 4 LESS #306,A,Westside
4,2016,PHO LITTLE SAIGON,A,Westside


In [234]:
#convert to CSV
westside_csv_table_df = westside_final_df.to_csv("westside_table.csv")

In [235]:
connection_string = (
f"root:PASSWORD!@localhost/etl_project")
engine = create_engine(f'mysql://{connection_string}')

In [238]:
engine.table_names()

['eastside', 'westside']

In [237]:
westside_finalTruc_df=westside_final_df.iloc[0:50]
westside_finalTruc_df.to_sql(
name='westside', con=engine,
if_exists='append', index=False)

In [179]:
#**********************************-----------EAST SIDE DATA------------------************************************

In [94]:
#verify data is pulled into jupyter
#NYC data is east coast
eastside_file = "../Resources/east_side.csv"
eastside_data_df = pd.read_csv(eastside_file)
#show data
eastside_data_df.head(5)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,6/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,30.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection
1,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.,Not Critical,NaN,NaN,NaN,8/28/2017,Administrative Miscellaneous / Initial Inspection
2,50046354,VITE BAR,QUEENS,2507,BROADWAY,11106.0,3478134702,Italian,10/3/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection
3,50061389,TACK'S CHINESE TAKE OUT,STATEN ISLAND,11C,HOLDEN BLVD,10314.0,7189839854,Chinese,5/17/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,46.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection
4,41516263,NO QUARTER,BROOKLYN,8015,5 AVENUE,11209.0,7187019180,American,3/30/2017,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,18.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection


In [97]:
#get year from date
eastside_data_df["Year"]=pd.DatetimeIndex(eastside_data_df["INSPECTION DATE"]).year
eastside_data_df.head(5)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Year
0,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,6/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,30.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection,2015
1,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.,Not Critical,NaN,NaN,NaN,8/28/2017,Administrative Miscellaneous / Initial Inspection,2014
2,50046354,VITE BAR,QUEENS,2507,BROADWAY,11106.0,3478134702,Italian,10/3/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection,2016
3,50061389,TACK'S CHINESE TAKE OUT,STATEN ISLAND,11C,HOLDEN BLVD,10314.0,7189839854,Chinese,5/17/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,46.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection,2017
4,41516263,NO QUARTER,BROOKLYN,8015,5 AVENUE,11209.0,7187019180,American,3/30/2017,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,18.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection,2017


In [98]:
#add column and add location
eastside_datalocation_df=eastside_data_df.assign(Location="Eastside")
eastside_datalocation_df.head(5)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Year,Location
0,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,6/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,30.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection,2015,Eastside
1,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.,Not Critical,NaN,NaN,NaN,8/28/2017,Administrative Miscellaneous / Initial Inspection,2014,Eastside
2,50046354,VITE BAR,QUEENS,2507,BROADWAY,11106.0,3478134702,Italian,10/3/2016,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection,2016,Eastside
3,50061389,TACK'S CHINESE TAKE OUT,STATEN ISLAND,11C,HOLDEN BLVD,10314.0,7189839854,Chinese,5/17/2017,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,46.0,NaN,NaN,8/28/2017,Pre-permit (Operational) / Initial Inspection,2017,Eastside
4,41516263,NO QUARTER,BROOKLYN,8015,5 AVENUE,11209.0,7187019180,American,3/30/2017,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,18.0,NaN,NaN,8/28/2017,Cycle Inspection / Initial Inspection,2017,Eastside


In [102]:
#need only date, business name and inspection grade
eastside_new_df = eastside_datalocation_df[["Year", "DBA", "GRADE", "Location"]]
eastside_new_df.head()

##chicago_crime = chicago_crime_df[["Case Number", "Date", "Description"]]
##chicago_crime.head(5)

,Year,DBA,GRADE,Location
0,2015,NOTARO RESTAURANT,NaN,Eastside
1,2014,NOTARO RESTAURANT,NaN,Eastside
2,2016,VITE BAR,NaN,Eastside
3,2017,TACK'S CHINESE TAKE OUT,NaN,Eastside
4,2017,NO QUARTER,NaN,Eastside


In [104]:
#drop blank data
eastside_drop_df=eastside_new_df.dropna(how='any', axis=0)
eastside_drop_df.head(5)

,Year,DBA,GRADE,Location
6,2017,HENRI'S BACKYARD,C,Eastside
7,2017,RICHMOND COUNTY COUNTRY CLUB,A,Eastside
9,2015,TOMOE SUSHI,A,Eastside
12,2016,CRAB SPOT RESTAURANT,A,Eastside
13,2017,M NOODLE SHOP,A,Eastside


In [120]:
#selecting dates 2015,2016,2017
date_range=(eastside_drop_df['Year']>2014) & (eastside_drop_df['Year']<2018)
eastside_final_df=eastside_drop_df[date_range]
eastside_final_df.tail(10)

,Year,DBA,GRADE,Location
399894,2017,BELLA'S PLACE,A,Eastside
399898,2016,MARIA COFFEE SHOP & RESTAURANT,C,Eastside
399899,2016,WETZEL'S PRETZELS,A,Eastside
399901,2015,MUG SHOTS,A,Eastside
399902,2015,SUBWAY,A,Eastside
399909,2016,THE LOCAL,A,Eastside
399910,2016,RICHNESS RESTAURANT,A,Eastside
399912,2017,DUNKIN' DONUTS,A,Eastside
399914,2016,MAGGIES PLACE,A,Eastside
399917,2016,DUNKIN' DONUTS,A,Eastside


In [113]:
#data statistics--ALL
east_final_stats_df=eastside_final_df.describe(include='all')
east_final_stats_df

,Year,DBA,GRADE,Location
count,156050.000000,156050,156050,156050
unique,NaN,20300,6,1
top,NaN,DUNKIN' DONUTS,A,Eastside
freq,NaN,2279,123973,156050
mean,2015.892336,NaN,NaN,NaN
std,0.781329,NaN,NaN,NaN
min,2015.000000,NaN,NaN,NaN
25%,2015.000000,NaN,NaN,NaN
50%,2016.000000,NaN,NaN,NaN
75%,2017.000000,NaN,NaN,NaN


In [164]:
#Rename columns
eastside_final_df.rename(index=str, columns={"DBA":"Business"}, inplace=True)
eastside_final_df.head(5)

C:\Users\niche\Anaconda3\envs\pythondata\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Year,Business,GRADE,Location
6,2017,HENRI'S BACKYARD,C,Eastside
7,2017,RICHMOND COUNTY COUNTRY CLUB,A,Eastside
9,2015,TOMOE SUSHI,A,Eastside
12,2016,CRAB SPOT RESTAURANT,A,Eastside
13,2017,M NOODLE SHOP,A,Eastside


In [176]:
#convert to CSV
eastside_csv_table_df = eastside_final_df.to_csv("eastside_table.csv")

In [187]:
connection_string = (
f"root:PASSWORD@localhost/etl_project")
engine = create_engine(f'mysql://{connection_string}')

In [188]:
engine.table_names()

['eastside']

In [192]:
eastside_finalTruc_df=eastside_final_df.iloc[0:50]
eastside_finalTruc_df.to_sql(
name='eastside', con=engine,
if_exists='append', index=False)